In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from tqdm import tqdm

In [ ]:
def Bellman_IS_2D(lamb_value, sigma_1, sigma_2, eta_1, eta_2, x0_1, x0_2, steps):
    reward = np.zeros((nb_stocks+1, nb_periods))
    policy = np.zeros((nb_stocks+1, nb_periods))
    
    for period in range(nb_periods-1, -1, -1):
        for stock in range(nb_stocks+1):
            psi = stock
            x = nb_stocks - psi
            
            if psi == nb_stocks:
                reward[psi, period] = 0
                policy[psi, period] = 0
            else:
                if period == nb_periods - 1:
                    reward[psi, period] = np.inf
                    policy[psi, period] = np.inf
                    
                elif period == nb_periods - 2:
                    reward[psi, period] = -1 * lamb * (sigma ** 2) * (x ** 2) - eta * (x ** 2)
                    policy[psi, period] = x
                else:
                    base_reward_calculation = -1 * lamb * sigma * (x ** 2)
                    r = base_reward_calculation + reward[psi, period+1]
                    p = 0
                    i = 1

                    while (x - i >= 0):
                        potential_r = base_reward_calculation + reward[psi + i, period + 1] - eta * (i ** 2)
                        if r < potential_r:
                            r = potential_r
                            p = i
                        i += 1
                        
                    reward[psi, period] = r
                    policy[psi, period] = p
                        
    trajectory = [nb_stocks]
    initial = 0
    
    for period in range(1, nb_periods + 1):
        instant_past = trajectory[-1]
        psi = int(policy[initial, period - 1])
        initial += psi
        new_remaining_inventory = instant_past - psi
        trajectory.append(new_remaining_inventory)

    return trajectory

In [ ]:
x0_1 = 100
x0_2 = 0

sigma_1 = 0.20
sigma_2 = 0.20
rho = 0.9

steps = 75

eta_1 = 5 * 1e-3
eta_2 = 5 * 1e-1
lambdas = [5e-4, 1e-3, 1e-2 , 1e-1, 1, 10]

In [ ]:
tab = {'t':[],'x_t':[],'lambda':[]}
for lamb_value in lambdas:
    traj = Bellman_IS_2D(lamb_value, sigma_1, sigma_2, eta_1, eta_2, x0_1, x0_2, steps)
    tab['t'].extend(range(len(traj)))
    tab['x_t'].extend(traj)
    tab['lambda'].extend([str(lamb_value) for i in range(len(traj))])

df = pd.DataFrame(tab)
fig = px.line(df, x='t', y='x_t', color='lambda', template='plotly_white', 
              labels={"x_t": "Remaining Inventory", "t": "Time", "lambda": "Market Impact"})

fig.update_traces(hoverinfo='skip')
fig.update_traces(hovertemplate=None)
fig.update_layout(width=900, height=300)
fig.show()